# **Notebook for Prompts (using Langchain, Outputparser, ...)**

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

import openai
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.prompts import ChatPromptTemplate

In [ ]:
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

llm_model = "gpt-3.5-turbo"

In [ ]:
def get_completion(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

get_completion("Hi",)

In [ ]:
chat = ChatOpenAI(temperature=0.0, model=llm_model)
chat

In [2]:
def read_txt(file_path):
    """
    Reads text from a file located at 'file_path' and returns it as a string.

    :param file_path: Path to the text file.
    :return: String containing the contents of the file.
    """
    try:
        with open(file_path, 'r') as file:
            return file.read()
    except IOError:
        return "Error: File not found or cannot be read."

## **Backend**

### Backend Dev

In [3]:
# prompt components
prompt_txt = read_txt('database_dev.txt')
database_dev_schema_1 = ResponseSchema(name="code",
                             description="SQL Statements you wrote for the task.")
orchestrator_db = "Schema for a newsletter signup page." # Example

# define ouput parser
response_schemas = [database_dev_schema_1]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

# make prompt 
prompt = ChatPromptTemplate.from_template(template=prompt_txt)
backend_dev_prompt = prompt.format_messages(format_instructions=format_instructions,
                                 orchestrator_db = orchestrator_db)

In [4]:
backend_dev_prompt_str = backend_dev_prompt[0].content
print(backend_dev_prompt_str)

You are a software developer specialized in databases. It is your responsibility to write correct and high quality SQL statements for different projects. You will be provided with several requirements your manager came up with based on some user request. It is now your task to provide the data definition language that fulfill these requirements. You should there for output the necessary SQL statements that create the tables for the database as well as any other SQL statements that are needed. These are the requirements:
Schema for a newsletter signup page.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"code": string  // SQL Statements you wrote for the task.
}
```


In [ ]:
# this is how a response would be parsed to get values
response = chat(backend_dev_prompt)
output_dict = output_parser.parse(response.content)
output_dict["code"]

### Backend Test

In [5]:
# prompt components
prompt_txt = read_txt('backend_test.txt')
database_test_schema_1 = ResponseSchema(name="revision_needed",
                                        description="Based on your evaluation, are there any errors that need to be fixed? Answer True if yes, False if no.")
database_test_schema_2 = ResponseSchema(name="feedback",
                                        description="Report with your analysis of the code.")

backend_code = "No code"
backend_logs = "No errors"

# define ouput parser
response_schemas = [database_test_schema_1,database_test_schema_2]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

# make prompt 
prompt = ChatPromptTemplate.from_template(template=prompt_txt)
backend_test_prompt = prompt.format_messages(format_instructions=format_instructions,
                                 backend_code = backend_code,
                                 backend_logs = backend_logs)

In [6]:
backend_test_prompt_str = backend_test_prompt[0].content
print(backend_test_prompt_str)

You are a software developer specialized in testing backend systems and it is your responsibility to check code for errors or mistakes. You will be provided with some code that is used to interact with a database as well as some error logs if there are any. 
This is the code you have to check:
No code

These are the error logs:
No errors

Based on this code write a report and point out any mistakes or errors you have found. Explain them and suggest how they could be corrected. Only point errors, mistakes or things that are incorrect and do not talk about things that are well done. It is possible that there are no errors or bad practices in which case your report will be empty.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"revision_needed": string  // Based on your evaluation, are there any errors that need to be fixed? Answer True if yes, False if no.
	"feedback": string  // Report wi

### Backend Doc

In [7]:
# prompt components
prompt_txt = read_txt('backend_doc.txt')
database_doc_schema_1 = ResponseSchema(name="documentation",
                                        description="The documentation of the code.")

backend_code = "No code"

# define ouput parser
response_schemas = [database_doc_schema_1]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

# make prompt 
prompt = ChatPromptTemplate.from_template(template=prompt_txt)
backend_doc_prompt = prompt.format_messages(format_instructions=format_instructions,
                                 backend_code = backend_code)

In [8]:
backend_doc_prompt_str = backend_doc_prompt[0].content
print(backend_doc_prompt_str)

You are a specialist for writing a documentation for code. It is your responsibility to analyze some given code and write a short description, so that someone else can understand what functionalities there are. You will be provided with some code that is used to access a database. It is your task to write a short documentation that describes what functions there are and what they are capable of. Explicitly point out what URL endpoints there are.
This is the code:
No code
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"documentation": string  // The documentation of the code.
}
```
